In [82]:
import numpy as np
from scipy.stats import norm
from scipy.optimize import minimize as minimise # we don't do US spelling
import pandas as pd

In [83]:
"""
Sales data should be unaffected by limits in storage. Sales data should tell us the mean and standard deviation of demand for the product.
"""
sales_data = pd.DataFrame({
    "product"       : ["salt","beans","juice","ink"],
    "profit"        : [10,20,15,100],
    "mean_demand"    : [500,300,650,50],
    "demand_sd"      : [50,30,100,12]
})

In [84]:
def conditional_mean(mu: float, sd: float, c: float) -> float:
    """
    Calculates E[X | X < c] for a normal distribution
    """
    numerator = mu * norm.cdf((c - mu)/sd) - sd * norm.pdf((c - mu)/sd)
    denominator = norm.cdf((c - mu)/sd)
    return numerator/denominator


def profit_contribution(row: pd.Series) -> float:
    """
    Calculates the expected profit generated by one product.
    """
    storage = row["storage"]
    sd = row["demand_sd"]
    mu = row["mean_demand"]
    profit = row["profit"]

    # probability we have enough of the product
    p_enough = norm.cdf(x=storage, loc=mu, scale=sd)

    # conditional expected sales given Sales < storage
    cond_e_sales = conditional_mean(mu=mu, sd=sd, c=storage)
    
    # expected net profit = profit per item * E[min(Sales, storage)]
    return profit * (p_enough * cond_e_sales + (1 - p_enough) * storage)


def expected_profit(storage_vals: list[float], sales_data: pd.DataFrame) -> float:
    """
    Computes total expected profit across all items
    """
    sales_data = sales_data.copy()
    sales_data["storage"] = storage_vals

    # calculate the expected profit of each item individually
    sales_data["expected_profit_contribution"] = sales_data.apply(profit_contribution, axis=1)
    # return sum
    return sales_data["expected_profit_contribution"].sum()

def objective(storage_vals: list[float], sales_data: pd.DataFrame) -> float:
    """
    Negative expected profit because we'll use a minimizer.
    """
    # Use your monte carlo function, or vectorized expected profit function
    return -expected_profit(storage_vals, sales_data)

def maximise_expected_profit(sales_data: pd.DataFrame, total_storage: float) -> list[float]:
    n_products = len(sales_data)

    # initial guess: equal allocation
    x0 = np.full(n_products, total_storage/n_products)

    # constraint: total storage allocations adds up to total allowed storage
    constraints = {
        "type":"eq",
        "fun": lambda x: np.sum(x)-total_storage
    }
    
    bounds = [(0,None) for i in range(n_products)]
    
    result = minimise(objective, x0, args=(sales_data,), method='SLSQP', bounds=bounds, constraints=constraints)
    if result.success:
        return result.x.tolist()
    else:
        raise ValueError("Optimisation failed" + result.message)
    

In [85]:
def monte_carlo_expected_profit(storage_vals: list[int], sales_data: pd.DataFrame, n_sim: int = 100_000) -> float:
    """
    Estimate total expected profit using Monte Carlo simulation.

    Parameters:
        storage_vals: list of storage levels for each product
        sales_data: DataFrame with columns "mean_demand", "demand_sd", "profit"
        n_sim: number of Monte Carlo samples

    Returns:
        Estimated total expected profit
    """
    total_expected_profit = 0.0

    for storage, row in zip(storage_vals, sales_data.itertuples(index=False)):
        mu = row.mean_demand
        sd = row.demand_sd
        profit = row.profit

        # simulate sales
        simulated_sales = np.random.normal(loc=mu, scale=sd, size=n_sim)
        # Apply storage limit
        units_sold = np.minimum(simulated_sales, storage)
        # Compute expected profit
        expected_profit = profit * units_sold.mean()

        total_expected_profit += expected_profit

    return total_expected_profit

In [86]:
maximise_expected_profit(sales_data, total_storage=2000)

[622.3372569962918, 380.5564556667722, 908.8790365029811, 88.22725083395505]